In [1]:
import pandas as pd, numpy as np
import imp, re

import z2_save_jaspar, z1_save_oligos, z3_transformations
from scipy.stats import ks_2samp
import scipy.stats as stats
idx = pd.IndexSlice

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [2]:

#LOAD JASPAR & OLIGO DATA
jaspar = z2_save_jaspar.load_jaspar()
oligos,oligos_by_exp = z1_save_oligos.load_oligos_plus()
oligos_by_exp["analysis_group_key"] = oligos_by_exp.exp.copy()
all_obe = oligos_by_exp.reset_index().loc[lambda x:x.mutant_num<5].groupby(["starts","mutant_num","analysis_group_key"]).norm_mu.mean()


In [3]:
print("HIHIHIH")

HIHIHIH


In [4]:
filters =  z3_transformations.compute_filters(all_obe, hq = False)


/usr/local/anaconda3/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [5]:
print(f"""{oligos_by_exp.n_bcs.sum()} barcodes,  {oligos_by_exp.n_transcripts.sum()} total UMIs. 
{oligos_by_exp.n_transcripts.sum() / oligos_by_exp.n_bcs.sum()} umis per barcode
{oligos_by_exp.groupby(["oligo","analysis_group_key"]).n_transcripts.sum().mean()}
""")



6698373 barcodes,  15778869 total UMIs. 
2.3556271052686975 umis per barcode
85.35023692068026



In [6]:
tile_sets_nmu = oligos_by_exp.groupby(["starts"]).mean().norm_mu
wt_tiles_nmu = oligos_by_exp.loc[lambda x: x.mutant_num == 0].groupby(["starts"]).mean().norm_mu

e5_tile_starts = oligos_by_exp.loc[lambda x: x.mutant_num == 0]\
    .groupby(["starts"]).mean().loc[lambda x: x.norm_mu >= x.norm_mu.quantile(.95) ].index
e1_tile_starts = oligos_by_exp.loc[lambda x: x.mutant_num == 0]\
    .groupby(["starts"]).mean().loc[lambda x: x.norm_mu > x.norm_mu.quantile(.99) ].index

print(f"""average normalized mu over all tile sets in all experiments {tile_sets_nmu.mean()}
average normalized my over all wildtype tiles {wt_tiles_nmu.mean()}

the 95th percentile value of enmu over all wildtype tiles is {wt_tiles_nmu.quantile(.95)}
the 99th percentile value of enmu over all wiltype tiles is {wt_tiles_nmu.quantile(.99)}

the 95th percentile value of enmu over all tiles is {tile_sets_nmu.quantile(.95)}
the 99th percentile value of enmu over all tiles is {tile_sets_nmu.quantile(.99)}

We will define the e5 and e1 groups using the average expression levels of wild type oligos
being over the 95th and 99th percentiles respectively.

{len(e5_tile_starts)} and {len(e1_tile_starts)} of {oligos_by_exp.starts.nunique()} were identified in 
the e5 and e1 sets respectively.
""")

average normalized mu over all tile sets in all experiments 2.29485067642086
average normalized my over all wildtype tiles 2.3237568953694314

the 95th percentile value of enmu over all wildtype tiles is 3.026028017721101
the 99th percentile value of enmu over all wiltype tiles is 6.562644394613727

the 95th percentile value of enmu over all tiles is 2.951008743282446
the 99th percentile value of enmu over all tiles is 5.371244117961561

We will define the e5 and e1 groups using the average expression levels of wild type oligos
being over the 95th and 99th percentiles respectively.

99 and 20 of 2091 were identified in 
the e5 and e1 sets respectively.



In [7]:
obe_by_start = oligos_by_exp.reset_index().set_index("starts").sort_index()

In [8]:

oligo_types =pd.concat([pd.concat([
obe_by_start.loc[max(mutant_start_position-150,0):mutant_start_position].loc[lambda x: x.mutant_start == mutant_start_position].assign(tile_class="ablation"),
obe_by_start.loc[max(mutant_start_position-150,0):mutant_start_position].loc[lambda x: (x.mutant_num > 0) &(x.mutant_num < 5) & (x.mutant_start != mutant_start_position)].assign(tile_class="other"),    
obe_by_start.loc[max(mutant_start_position-150,0):max(0,mutant_start_position-30)].loc[lambda x: x.mutant_num ==0].assign(tile_class="wildtype"),
obe_by_start.loc[max(mutant_start_position-150,0):mutant_start_position+30].loc[lambda x: (x.mutant_start == mutant_start_position) | 
                                                                                ( (x.mutant_start+30) == mutant_start_position)].assign(tile_class="ablation60"),    
obe_by_start.loc[max(mutant_start_position-150,0):mutant_start_position+30].loc[lambda x: (x.mutant_num > 0) &(x.mutant_num < 5) &
                                                                                ( (x.mutant_start != mutant_start_position) & 
                                                                                ( (x.mutant_start+30) != mutant_start_position))].assign(tile_class="other60"),    
obe_by_start.loc[max(mutant_start_position-150,0):max(0,mutant_start_position)].loc[lambda x: x.mutant_num ==0].assign(tile_class="wildtype60"),
    
    ], ignore_index =True).assign(mutant_tile_start = mutant_start_position) for mutant_start_position in oligos_by_exp.mutant_start.unique()]).set_index(["mutant_tile_start","tile_class"])


In [9]:
oligo_types["starts"] = oligo_types.join(oligos_by_exp.reset_index().drop_duplicates("oligo").set_index("oligo")[["starts"]],on="oligo").starts

In [10]:
oligo_types = oligo_types.sort_index(level=[0,1])

In [11]:

#means = pd.Series(mutant_starts).apply(lambda s: oligo_types.loc[idx[s,:],:].groupby("tile_class").norm_mu.mean())

def get_stats(s):
    tile_groups = oligo_types.loc[idx[s,:],:].groupby("tile_class")
    if not set(["ablation","other","wildtype",
                "ablation60","other60","wildtype60"]).issubset(tile_groups.groups.keys()): 
        print (f"skipping position {s}")
        return pd.Series().rename(s)

    ablations = tile_groups.get_group("ablation")
    others = tile_groups.get_group("other")
    wildtypes = tile_groups.get_group("wildtype")
    
    ab1_dld1 = tile_groups.get_group("ablation").loc[lambda x: x.exp=="DLD1_WT_BR1"]
    ot1_dld1 = tile_groups.get_group("other").loc[lambda x: x.exp=="DLD1_WT_BR1"]
    wt1_dld1 = tile_groups.get_group("wildtype").loc[lambda x: x.exp=="DLD1_WT_BR1"]
    
    ab2_dld1 = tile_groups.get_group("ablation").loc[lambda x: x.exp=="DLD1_WT_BR2"]
    ot2_dld1 = tile_groups.get_group("other").loc[lambda x: x.exp=="DLD1_WT_BR2"]
    wt2_dld1 = tile_groups.get_group("wildtype").loc[lambda x: x.exp=="DLD1_WT_BR2"]


    ablations60 = tile_groups.get_group("ablation60")
    others60 = tile_groups.get_group("other60")
    wildtypes60 = tile_groups.get_group("wildtype60")
    
    wt_tt = stats.ttest_ind( ablations.norm_mu, wildtypes.norm_mu)
    all_tt = stats.ttest_ind( ablations.norm_mu, pd.concat([wildtypes.norm_mu,others.norm_mu]))
    
    r1_tt = stats.ttest_ind( ab1_dld1.norm_mu, wt1_dld1.norm_mu)
    r2_tt = stats.ttest_ind( ab2_dld1.norm_mu, wt2_dld1.norm_mu)
    
    r1_aott = stats.ttest_ind( ab1_dld1.norm_mu, pd.concat([wt1_dld1.norm_mu,ot1_dld1.norm_mu]))
    r2_aott = stats.ttest_ind( ab2_dld1.norm_mu, pd.concat([wt2_dld1.norm_mu,ot2_dld1.norm_mu]))
            
        
#     wt_rep_tt = stats.ttest_ind( ablations.norm_mu, wildtypes.norm_mu)
#     all_rep_tt = stats.ttest_ind( ablations.norm_mu, pd.concat([wildtypes.norm_mu,others.norm_mu]))
    
    
    return pd.Series({

        "mu_wt":wildtypes.norm_mu.mean(),
        "mu_ot":others.norm_mu.mean(),
        "mu_ab":ablations.norm_mu.mean(),

        "mu_wt60":wildtypes60.norm_mu.mean(),
        "mu_ot60":others60.norm_mu.mean(),
        "mu_ab60":ablations60.norm_mu.mean(),
        
        "tt_wt_pval":-1 * np.log(wt_tt[1]),
        "tt_all_pval":-1 * np.log(all_tt[1]),
        
                
        "tt_wt_ttstat":wt_tt[0],
        "tt_all_ttstat":all_tt[0],
        
                        
        "tt_r1_ttstat":r1_tt[0],
        "tt_r2_ttstat":r2_tt[0],
                                
        "tt_r1_aottstat":r1_aott[0],
        "tt_r2_aottstat":r2_aott[0],
        
        "tt_r1_aott_pval":-1 * np.log(r1_aott[1]),
        "tt_r2_aott_pval":-1 * np.log(r2_aott[1]),
        
                

    }).rename(s)





In [12]:
stats =pd.Series(oligos_by_exp.mutant_start.unique()).apply(get_stats)

/usr/local/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3506: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)


skipping position 1526
skipping position 8765
skipping position 18665
skipping position 27323
skipping position 28482
skipping position 33406
skipping position 36590
skipping position 37858
skipping position 39927
skipping position 45650
skipping position 48608
skipping position 51607
skipping position 52516
skipping position 55123
skipping position 56239
skipping position 64877
skipping position 65976
skipping position 76831
skipping position 77697
skipping position 79793
skipping position 82883
skipping position 83130
skipping position 88519
skipping position 90394
skipping position 128438
skipping position 144036
skipping position 153383
skipping position 159713
skipping position 163665
skipping position 168686
skipping position 170376
skipping position 172627
skipping position 221467
skipping position 235292
skipping position 247170
skipping position 248652
skipping position 250023
skipping position 252768
skipping position 253678
skipping position 254312
skipping position 257239
s

In [13]:
stats.to_csv("../out/0107_dm_stats.csv")

In [ ]:
import seaborn as sns
%matplotlib inline

In [16]:
stats

,mu_wt,mu_ot,mu_ab,mu_wt60,mu_ot60,mu_ab60,tt_wt_pval,tt_all_pval,tt_wt_ttstat,tt_all_ttstat,tt_r1_ttstat,tt_r2_ttstat,tt_r1_aottstat,tt_r2_aottstat,tt_r1_aott_pval,tt_r2_aott_pval
0,2.593088,2.099656,2.593088,2.593088,2.204640,2.593088,-0.000000,5.660703,0.000000,2.997684,NaN,NaN,NaN,NaN,NaN,NaN
1,2.236180,2.270404,2.272616,2.394251,2.676935,2.243921,0.417801,0.099907,0.442754,0.119508,0.683552,-0.055595,0.917390,0.228596,1.003742,0.197492
2,2.593088,2.228389,2.167999,2.445344,2.256119,2.309695,8.468994,1.646785,-4.026271,-1.308309,NaN,NaN,-0.334532,-1.149909,0.292252,1.260918
3,2.445344,2.237635,2.198331,2.335440,2.190170,2.186198,5.275099,1.340371,-2.880974,-1.124938,-0.127059,0.049968,-0.157250,0.619600,0.130720,0.604718
4,2.335440,2.196954,2.188582,2.332771,2.133607,2.192760,2.350881,0.500704,-1.682620,-0.516156,0.294293,-0.241964,0.319052,-0.097038,0.283213,0.079294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
